# ZeroPM Tutorial

This notebook demonstrates the functionality of the ZeroPM class, which provides efficient access to the ZeroPM SQLite database containing chemical identifiers and properties.

## Overview

ZeroPM allows you to:
- Query chemicals by CAS number or name
- Query chemicals by regulatory inventory, country, or region
- Convert between different chemical identifiers (CAS, InChI, InChIKey, SMILES)
- Perform fuzzy name searches
- Batch process multiple chemicals
- Search by substructure
- Export results to CSV

## Database Auto-Download

The ZeroPM database (~400MB) is automatically downloaded from [GitHub](https://github.com/ZeroPM-H2020/global-chemical-inventory-database) on first use. The download happens once and the database is cached locally for future use.

Let's get started!

## 1. Installation and Setup

First, make sure you have the required dependencies installed.

In [1]:
# Import the ZeroPM class
from provesid.zeropm import ZeroPM

# Initialize the ZeroPM database connection
zpm = ZeroPM()

print("✓ ZeroPM initialized successfully!")
print(f"Database path: {zpm.db_path}")

✓ ZeroPM initialized successfully!
Database path: C:\projects\git\PROVESID\src\provesid\data\zeropm-v0-0-3.sqlite


## 2. Database Statistics

Let's start by exploring what's in the database.

In [2]:
# Get database statistics
stats = zpm.get_database_stats()

print("Database Statistics:")
print("=" * 50)
for key, value in stats.items():
    print(f"{key:30s}: {value:>15,}" if isinstance(value, int) else f"{key:30s}: {value}")

Database Statistics:
api_ready_query               :         447,617
api_results                   :       1,349,825
substances                    :         359,221
inventories                   :         543,851
inventory_summary             :       1,045,887
cleanventory_chemicals        :         483,388
zeropm_chemicals              :         126,369
components                    :          22,380
multi_components              :          38,696
unique_cas_numbers            :         164,513
unique_chemical_names         :         283,104


## 3. Querying by CAS Number

The most common way to query the database is using a CAS Registry Number.

In [3]:
# Get a sample CAS number from the database
zpm.cursor.execute("""
    SELECT query 
    FROM api_ready_query 
    WHERE type = 'CAS Registry Number' 
    LIMIT 1
""")
sample_cas = zpm.cursor.fetchone()[0]

print(f"Sample CAS number: {sample_cas}")

# Query the CAS number to get a query_id
query_id = zpm.query_cas(sample_cas)
print(f"Query ID: {query_id}")

# Get InChI information
inchi_ids, ranks = zpm.get_inchi_id(query_id)
if inchi_ids:
    inchi, inchikey = zpm.get_inchi(inchi_ids[0])
    print(f"\nInChI: {inchi[:50]}..." if len(inchi) > 50 else f"\nInChI: {inchi}")
    print(f"InChIKey: {inchikey}")

Sample CAS number: 121-20-0
Query ID: 1

InChI: InChI=1S/C21H28O5/c1-7-8-9-14-13(3)17(11-16(14)22)...
InChIKey: SHCRDCOTRILILT-WOBDGSLYSA-N


## 4. Converting CAS to SMILES

ZeroPM can convert CAS numbers to SMILES using RDKit.

In [4]:
# Get SMILES from CAS number
smiles = zpm.get_smiles_from_cas(sample_cas)
print(f"CAS: {sample_cas}")
print(f"SMILES: {smiles}")

# Get chemical names
names = zpm.get_names(sample_cas)
if names:
    print(f"\nAlternative names ({len(names)}):")
    for i, name in enumerate(names[:5], 1):  # Show first 5 names
        print(f"  {i}. {name}")
    if len(names) > 5:
        print(f"  ... and {len(names) - 5} more")

CAS: 121-20-0
SMILES: C/C=C\CC1=C(C)[C@@H](OC(=O)[C@@H]2[C@@H](/C=C(\C)C(=O)OC)C2(C)C)CC1=O

Alternative names (7):
  1. Cyclopropanecarboxylic acid, 3-[(1E)-3-methoxy-2-methyl-3-oxo-1-propenyl]-2,2-dimethyl-, (1S)-3-(2Z)-2-butenyl-2-methyl-4-oxo-2-cyclopenten-1-yl ester, (1R,3R)-
  2. 3-(but-2-enyl)-2-methyl-4-oxocyclopent-2-enyl 2,2-dimethyl-3-(3-methoxy-2-methyl-3-oxoprop-1-enyl)cyclopropanecarboxylate
  3. 3-(but-2-enyl)-2-methyl-4-oxocyclopent-2-enyl2,2-dimethyl-3-(3-methoxy-2-methyl-3-oxoprop-1-enyl)cyclopropanecarboxylate
  4. CINERIN II
  5. Jasmolin II
  ... and 2 more


## 5. Querying by Chemical Name

You can search for chemicals by their exact name or use fuzzy matching.

In [6]:
# Get a sample chemical name
zpm.cursor.execute("""
    SELECT query 
    FROM api_ready_query 
    WHERE type = 'chemical name' 
    LIMIT 1
""")
sample_name = zpm.cursor.fetchone()[0]

# Exact match
query_id = zpm.query_name(sample_name)
print(f"Exact search for '{sample_name}'")
print(f"Query ID: {query_id}")

# Fuzzy match (with partial name)
if len(sample_name) >= 5:
    partial_name = sample_name[:5]
    print(f"\nFuzzy search for '{partial_name}':")
    similar_ids = zpm.query_similar_name(partial_name, number_of_results=5, score_cutoff=70)
    if similar_ids:
        print(f"Found {len(similar_ids)} similar matches")
        for qid in similar_ids[:3]:
            zpm.cursor.execute("SELECT query FROM api_ready_query WHERE query_id = ?", (qid,))
            result = zpm.cursor.fetchone()
            if result:
                print(f"  - {result[0]}")

Exact search for 'cinerin II'
Query ID: 2

Fuzzy search for 'ciner':
Found 5 similar matches
  - cinerin II
  - cinerin I
  - ne


## 6. Converting Between Different Identifiers

ZeroPM supports conversion between various chemical identifiers.

In [7]:
# Get a sample InChI and InChIKey
zpm.cursor.execute("""
    SELECT inchi, inchikey 
    FROM substances 
    LIMIT 1
""")
test_inchi, test_inchikey = zpm.cursor.fetchone()

print("Identifier Conversions:")
print("=" * 60)

# InChI to CAS
cas_from_inchi = zpm.get_cas_from_inchi(test_inchi)
print(f"InChI → CAS: {cas_from_inchi}")

# InChIKey to CAS
cas_from_key = zpm.get_cas_from_inchikey(test_inchikey)
print(f"InChIKey → CAS: {cas_from_key}")

# InChIKey to SMILES
smiles_from_key = zpm.get_smiles_from_inchikey(test_inchikey)
print(f"InChIKey → SMILES: {smiles_from_key}")

# SMILES to CAS (if we have a valid SMILES)
if smiles_from_key:
    cas_from_smiles = zpm.get_cas_from_smiles(smiles_from_key)
    print(f"SMILES → CAS: {cas_from_smiles}")

Identifier Conversions:
InChI → CAS: ['100-00-5', '68239-23-6']
InChIKey → CAS: ['100-00-5', '68239-23-6']
InChIKey → SMILES: O=[N+]([O-])c1ccc(Cl)cc1
SMILES → CAS: ['100-00-5', '68239-23-6']


[22:07:01] WARNING: Charges were rearranged



## 7. Batch Processing

For efficiency, ZeroPM provides batch methods to process multiple chemicals at once.

In [8]:
# Get multiple CAS numbers
zpm.cursor.execute("""
    SELECT query 
    FROM api_ready_query 
    WHERE type = 'CAS Registry Number' 
    LIMIT 5
""")
cas_list = [row[0] for row in zpm.cursor.fetchall()]

print(f"Batch processing {len(cas_list)} CAS numbers:")
print("=" * 60)

# Batch query CAS numbers
query_ids = zpm.batch_query_cas(cas_list)
for cas, qid in query_ids.items():
    print(f"{cas}: Query ID = {qid}")

print("\n" + "=" * 60)

# Batch get SMILES
smiles_dict = zpm.batch_get_smiles_from_cas(cas_list)
for cas, smiles in smiles_dict.items():
    print(f"{cas}: {smiles if smiles else 'N/A'}")

Batch processing 5 CAS numbers:
121-20-0: Query ID = 1
25646-71-3: Query ID = 4
76823-93-3: Query ID = 9
177964-68-0: Query ID = 11
27955-94-8: Query ID = 14

121-20-0: C/C=C\CC1=C(C)[C@@H](OC(=O)[C@@H]2[C@@H](/C=C(\C)C(=O)OC)C2(C)C)CC1=O
25646-71-3: CCN(CCNS(C)(=O)=O)c1ccc(N)c(C)c1.CCN(CCNS(C)(=O)=O)c1ccc(N)c(C)c1.O=S(=O)(O)O.O=S(=O)(O)O.O=S(=O)(O)O
76823-93-3: N#CCCSCc1csc(NC(=N)N)n1
177964-68-0: COCc1c(C(C)C)nc(C(C)C)c(C=CC=O)c1-c1ccc(F)cc1
27955-94-8: CC(c1ccc(O)cc1)(c1ccc(O)cc1)c1ccc(O)cc1


## 8. Batch InChIKey to CAS Conversion

In [9]:
# Get multiple InChIKeys
zpm.cursor.execute("""
    SELECT inchikey 
    FROM substances 
    LIMIT 5
""")
inchikey_list = [row[0] for row in zpm.cursor.fetchall()]

print(f"Batch converting {len(inchikey_list)} InChIKeys to CAS:")
print("=" * 60)

cas_dict = zpm.batch_get_cas_from_inchikey(inchikey_list)
for key, cas in cas_dict.items():
    print(f"{key}: {cas if cas else 'N/A'}")

Batch converting 5 InChIKeys to CAS:
: 71889-03-7
AAADGWUCZIMSKQ-UHFFFAOYSA-M: 16509-22-1
AAADKYXUTOBAGS-UHFFFAOYSA-N: 78-99-9
AAAFFJJBQGZTFF-UHFFFAOYSA-N: 5355-88-4
AAAFYYTUBLYGNG-UHFFFAOYSA-N: N/A


## 9. Advanced Search: Regex Pattern Matching

Search for chemicals using pattern matching.

In [10]:
# Search for chemicals with names containing a pattern
# For example, search for names containing "acid"
pattern = "%acid%"

results = zpm.query_name_regex(pattern, case_sensitive=False, limit=10)

print(f"Chemical names matching pattern '{pattern}':")
print("=" * 60)
for query_id, name in results[:5]:
    print(f"  {name}")
print(f"\nTotal matches: {len(results)}")

Chemical names matching pattern '%acid%':
  2-chloroethylphosphonic acid
  Isocyanic acid, 2-methyl-m-phenylene ester
  3-(4-aminophenyl)-2-cyano-2-propenoic acid
  5-{4-[5-5-amino-2-[4-(2-sulfoxyethylsulfonyl)phenylazo]-4-sulfo-phenylamino]-6-chloro-1,3,5-triazin-2-ylamino}}-4-hydroxy-3-(1-sulfo-naphthalen-2-ylazo)-naphthalene-2,7-disulfonicacid sodium salt
  acetic acid ... %

Total matches: 10


## 10. Advanced Search: Substructure Search

Search for chemicals containing a specific substructure (SMARTS pattern).

⚠️ **Note:** This operation can be slow for large searches as it needs to check each molecule.

In [11]:
# Search for molecules containing a benzene ring
smarts_pattern = "c1ccccc1"  # Benzene ring

print(f"Searching for molecules with benzene ring (max 5 results)...")
results = zpm.get_cas_by_substructure(smarts_pattern, max_results=5)

print(f"Found {len(results)} molecules with benzene ring:")
print("=" * 60)
for i, compound in enumerate(results, 1):
    print(f"\n{i}. CAS: {compound['cas']}")
    print(f"   SMILES: {compound['smiles']}")
    print(f"   InChIKey: {compound['inchikey'][:27]}...")

Searching for molecules with benzene ring (max 5 results)...
Found 5 molecules with benzene ring:

1. CAS: ['100-00-5', '68239-23-6']
   SMILES: O=[N+]([O-])c1ccc(Cl)cc1
   InChIKey: CZGCEKJOLUNIFY-UHFFFAOYSA-N...

2. CAS: ['100-01-6', '10040-98-9', '68239-24-7']
   SMILES: Nc1ccc([N+](=O)[O-])cc1
   InChIKey: TYMLOMAKGOJONV-UHFFFAOYSA-N...

3. CAS: ['100-02-7', '25154-55-6']
   SMILES: O=[N+]([O-])c1ccc(O)cc1
   InChIKey: BTJIUGUIPKRLHP-UHFFFAOYSA-N...

4. CAS: 100-03-8
   SMILES: O=S(O)c1ccc(Cl)cc1
   InChIKey: AOQYAMDZQAEDLO-UHFFFAOYSA-N...

5. CAS: ['100-04-9', '13533-17-0', '24564-52-1']
   SMILES: CN(C)c1ccc([N+]#N)cc1.[Cl-]
   InChIKey: CCIAVEMREXZXAK-UHFFFAOYSA-M...


## 11. Exporting Data to CSV

You can export query results to CSV files for further analysis.

In [12]:
import tempfile
import os

# Create a temporary directory for exports
temp_dir = tempfile.mkdtemp()

# Export batch results
output_file = os.path.join(temp_dir, 'cas_smiles_export.csv')
zpm.export_to_csv(
    list(smiles_dict.items()), 
    output_file,
    columns=['CAS', 'SMILES']
)

print(f"✓ Exported data to: {output_file}")

# Export custom query results
sql_query = """
    SELECT aq.query AS CAS, s.inchikey
    FROM api_ready_query aq
    JOIN api_results ar ON aq.query_id = ar.query_id
    JOIN substances s ON ar.inchi_id = s.inchi_id
    WHERE aq.type = 'CAS Registry Number'
    LIMIT 10
"""
output_file2 = os.path.join(temp_dir, 'cas_inchikey_export.csv')
zpm.export_query_results(sql_query, output_file2, include_headers=True)

print(f"✓ Exported custom query to: {output_file2}")

# List exported files
print(f"\nExported files in {temp_dir}:")
for file in os.listdir(temp_dir):
    filepath = os.path.join(temp_dir, file)
    size = os.path.getsize(filepath)
    print(f"  - {file} ({size:,} bytes)")

✓ Exported data to: C:\Users\aliak\AppData\Local\Temp\tmplwodoa9s\cas_smiles_export.csv
✓ Exported custom query to: C:\Users\aliak\AppData\Local\Temp\tmplwodoa9s\cas_inchikey_export.csv

Exported files in C:\Users\aliak\AppData\Local\Temp\tmplwodoa9s:
  - cas_inchikey_export.csv (400 bytes)
  - cas_smiles_export.csv (353 bytes)


## 12. Performance Optimization: Creating Indexes

Create database indexes to speed up queries.

In [13]:
# Create indexes for better query performance
print("Creating database indexes...")
index_results = zpm.create_indexes()

print("\nIndex Status:")
print("=" * 60)
for index_name, status in index_results.items():
    print(f"{index_name:30s}: {status}")

print("\n✓ Indexes created successfully!")
print("Note: Subsequent queries will be faster with these indexes.")

Creating database indexes...

Index Status:
idx_query                     : exists
idx_type                      : exists
idx_query_id_results          : exists
idx_inchi_id_results          : exists
idx_inchi                     : exists
idx_inchikey                  : exists
idx_inventory_query           : exists
idx_inventory_id              : exists

✓ Indexes created successfully!
Note: Subsequent queries will be faster with these indexes.


## 13. Creating Custom Views

Create database views for frequently used queries.

In [14]:
# Create a view for CAS to InChI mapping
view_name = "cas_to_inchi_view"
sql_query = """
    SELECT aq.query AS cas, s.inchi, s.inchikey
    FROM api_ready_query aq
    JOIN api_results ar ON aq.query_id = ar.query_id
    JOIN substances s ON ar.inchi_id = s.inchi_id
    WHERE aq.type = 'CAS Registry Number' AND ar.rank = 1
"""

success = zpm.create_view(view_name, sql_query)

if success:
    print(f"✓ View '{view_name}' created successfully!")
    
    # Query the view
    zpm.cursor.execute(f"SELECT * FROM {view_name} LIMIT 5")
    print(f"\nSample data from view:")
    print("=" * 60)
    for row in zpm.cursor.fetchall():
        print(f"CAS: {row[0]}, InChIKey: {row[2][:27]}...")
    
    # Clean up - drop the view
    zpm.cursor.execute(f"DROP VIEW IF EXISTS {view_name}")
    zpm.conn.commit()
    print(f"\n✓ View dropped for cleanup")
else:
    print(f"✗ Failed to create view")

✓ View 'cas_to_inchi_view' created successfully!

Sample data from view:
CAS: 121-20-0, InChIKey: SHCRDCOTRILILT-WOBDGSLYSA-N...
CAS: 121-20-0, InChIKey: SHCRDCOTRILILT-WOBDGSLYSA-N...
CAS: 121-20-0, InChIKey: SHCRDCOTRILILT-WOBDGSLYSA-N...
CAS: 121-20-0, InChIKey: SHCRDCOTRILILT-WOBDGSLYSA-N...
CAS: 25646-71-3, InChIKey: NPDFXFLCEDDWEG-UHFFFAOYSA-N...

✓ View dropped for cleanup


## 14. Complete Example: Workflow for Multiple Chemicals

Here's a complete workflow demonstrating how to process multiple chemicals efficiently.

In [15]:
import pandas as pd

# Get a sample of CAS numbers to process
zpm.cursor.execute("""
    SELECT query 
    FROM api_ready_query 
    WHERE type = 'CAS Registry Number' 
    LIMIT 10
""")
cas_numbers = [row[0] for row in zpm.cursor.fetchall()]

print(f"Processing {len(cas_numbers)} chemicals...")
print("=" * 80)

# Batch get all the data we need
query_ids = zpm.batch_query_cas(cas_numbers)
smiles_data = zpm.batch_get_smiles_from_cas(cas_numbers)
names_data = zpm.batch_get_names(cas_numbers)

# Create a pandas DataFrame
data = []
for cas in cas_numbers:
    data.append({
        'CAS': cas,
        'Query_ID': query_ids.get(cas),
        'SMILES': smiles_data.get(cas),
        'Names_Count': len(names_data.get(cas, [])),
        'First_Name': names_data.get(cas, [''])[0] if names_data.get(cas) else ''
    })

df = pd.DataFrame(data)

print("\nResults Summary:")
print(df.to_string(index=False, max_colwidth=50))

print(f"\n✓ Processed {len(cas_numbers)} chemicals successfully!")
print(f"  - {df['SMILES'].notna().sum()} have SMILES")
print(f"  - {df[df['Names_Count'] > 0].shape[0]} have alternative names")

Processing 10 chemicals...

Results Summary:
        CAS  Query_ID                                             SMILES  Names_Count                                         First_Name
   121-20-0         1 C/C=C\CC1=C(C)[C@@H](OC(=O)[C@@H]2[C@@H](/C=C(\...            7 Cyclopropanecarboxylic acid, 3-[(1E)-3-methoxy-...
 25646-71-3         4 CCN(CCNS(C)(=O)=O)c1ccc(N)c(C)c1.CCN(CCNS(C)(=O...           10 4-(N-ethyl-N-2-methanesulfonylaminoethyl)-2-met...
 76823-93-3         9                            N#CCCSCc1csc(NC(=N)N)n1            3 3-(2-(Diaminometyleeniamino)tiatsoli-4-yylimety...
177964-68-0        11      COCc1c(C(C)C)nc(C(C)C)c(C=CC=O)c1-c1ccc(F)cc1            4                                 (E)-crotonaldehyde
 27955-94-8        14            CC(c1ccc(O)cc1)(c1ccc(O)cc1)c1ccc(O)cc1            7           4,4',4''-(etaani-1,1,1-triyyli)trifenoli
122886-55-9        16 CCCCCCCCN=C(O)Nc1ccc(Cc2ccc(NC(O)=NCCCCCCCC)cc2...            8 Urea, N-octyl-N'-[4-[[4-[[(octylamino)carbonyl]

## 15. Summary and Best Practices

### Key Features:
1. **Simple Queries**: `query_cas()`, `query_name()`
2. **Fuzzy Matching**: `query_similar_name()` with configurable score cutoff
3. **Identifier Conversion**: Convert between CAS, InChI, InChIKey, and SMILES
4. **Batch Operations**: Process multiple chemicals efficiently
5. **Advanced Search**: Regex patterns and substructure matching
6. **Export**: Save results to CSV for further analysis
7. **Performance**: Create indexes for faster queries

### Best Practices:
- Use batch methods when processing multiple chemicals
- Create indexes before running many queries
- Use fuzzy matching for user input with potential typos
- Set appropriate score cutoffs for fuzzy matching (70-90 is typical)
- Export results to CSV for sharing or further analysis

### Performance Tips:
- Batch operations are much faster than individual queries
- Create indexes once at the start if doing many queries
- Use `score_cutoff` parameter to limit fuzzy search results
- Limit substructure searches with `max_results` parameter

## Conclusion

This notebook has demonstrated the main features of the ZeroPM class:

✓ Querying by CAS number and chemical name  
✓ Fuzzy name matching for handling variations  
✓ Converting between chemical identifiers  
✓ Batch processing for efficiency  
✓ Advanced searches (regex, substructure)  
✓ Exporting results to CSV  
✓ Performance optimization with indexes  

The ZeroPM class provides a convenient Python interface to the ZeroPM database, making it easy to work with chemical identifiers in your research and applications.

For more information, see the [PROVESID documentation](https://usetox.github.io/PROVESID/).